## Stargate Examples
This notebook demonstrates how to use the Stargate Document APIs on Astra.


### Basic Python Set-Up

In [ ]:
from pprint import pprint    # for pretty formatting
import requests              # for making REST API requests
import json                  # for converting json payloads to strings
import uuid                  # to create UUIDs for Astra connections
import os                    # for accessing creds
import warnings
warnings.filterwarnings('ignore') # to keep this notebook clean

In [ ]:
## Missing a package? No problem. You can import it directly in Jupyter
## Uncomment the two lines below to Install a pip package in the current Jupyter kernel
#import sys
#!{sys.executable} -m pip install requests

### A Helper API Client for Stargate

In [ ]:
class Client:
    """
    An API Client for connecting to Stargate
    """
    def __init__(self, base_url, access_token, headers):
        self.base_url = base_url
        self.access_token = access_token
        self.headers = headers    
    
    def post(self, payload={}, path=""):
        """
            Via the requests library, performs a post with the payload to the path
        """
        return requests.post(self.base_url + path, 
                             data=json.dumps(payload),
                             headers=self.headers)
    
    def put(self, payload={}, path=""):
        """
            Via the requests library, performs a put with the payload to the path
        """
        return requests.put(self.base_url + path, 
                            data=json.dumps(payload),
                            headers=self.headers)
    
    def patch(self, payload={}, path=""):
        """
            Via the requests library, performs a patch with the payload to the path
        """
        return requests.patch(self.base_url + path, 
                              data=json.dumps(payload),
                              headers=self.headers)
    
    def get(self, payload={}, path=""):
        """
            Via the requests library, performs a get with the payload to the path
        """
        return requests.get(self.base_url + path, 
                            data=json.dumps(payload),
                            headers=self.headers)
    
    def delete(self, payload={}, path=""):
        """
            Via the requests library, performs a delete with the payload to the path
        """
        return requests.delete(self.base_url + path, 
                             data=json.dumps(payload),
                             headers=self.headers)

#### Authentication has its own method because it still uses the v1 API

In [ ]:
def authenticate(path="/api/rest/v1/auth"):
    """
        This convenience function uses the v1 auth REST API to get an access token
        returns: an auth token; 30 minute expiration
    """
    url = BASE_URL + path # we still have to auth with the v1 API
    payload = {"username": USER,
               "password": PASSWORD}
    headers = {'accept': '*/*',
               'content-type': 'application/json',
               'x-cassandra-request-id': UUID}
    # make auth request to Astra
    r = requests.post(url, 
                      data=json.dumps(payload), 
                      headers=headers)
    # raise any authentication errror
    if r.status_code != 200:
        raise Exception(r.text)   
    # extract and return the auth token 
    data = json.loads(r.text)
    return data["authToken"]

### Connect to your Stargate Cluster in Astra

In [ ]:
# Astra globals
UUID = str(uuid.uuid1())
USER = os.environ["ASTRA_DB_USERNAME"]     # NEVER store your creds directly in your code!
PASSWORD = os.environ["ASTRA_DB_PASSWORD"] # NEVER store your creds directly in your code!
DB_ID = os.environ["ASTRA_DB_ID"]          # NEVER store your creds directly in your code!
REGION = os.environ["ASTRA_DB_REGION"]     # NEVER store your creds directly in your code!
REGION = "us-east1"
BASE_URL = f"https://{DB_ID}-{REGION}.apps.astra.datastax.com"

In [ ]:
TOKEN = authenticate()
HEADERS = {'content-type': 'application/json',
           'x-cassandra-token': TOKEN}
stargate_client = Client(BASE_URL, TOKEN, HEADERS)

## Create a table and post your document
#### --> The keyspace was set up during db creation.

In [ ]:
NAMESPACE = "battlestax"
COLLECTION = "games"
GAME_ID = str(uuid.uuid1())
DOC_ROOT_PATH = f"/api/rest/v2/namespaces/{NAMESPACE}/collections/{COLLECTION}/{GAME_ID}"

In [ ]:
game_doc = {
      "gameCode": "DANG",
      "currentState": {
        "name": "ADD_PLAYERS",
        "roundId": "",
      },
      "audienceSize": 0,
      "rounds": {
        1: {
          "type": "QUESTION",
          "title": "Round 1",
          "scoreMultiplier": 1,
        },
        2: {
          "type": "QUESTION",
          "title": "Round 1",
          "scoreMultiplier": 2,
        },
        3: {
          "type": "COMIC",
          "title": "Final Round",
          "scoreMultiplier": 3,
        },
      },
      "questions": {
        str(uuid.uuid1()): {
          "roundId": 1,
          "content": "What time is it?",
        },
        str(uuid.uuid1()): {
          "roundId": 2,
          "content": "What day is it?",
        },
        str(uuid.uuid1()): {
          "roundId": 3,
          "content": "https://xkcd.com/386/",
        },
      },
      "players": {}, 
      "answers": {}, 
      "votes": {}, 
      "audienceVotes": False
}

In [ ]:
response = stargate_client.put(game_doc, DOC_ROOT_PATH)
pprint(response.text)

### Get the full game

In [ ]:
response = stargate_client.get({}, DOC_ROOT_PATH)
pprint(response.json())

### Add Players to the Game

In [ ]:
players = {
      str(uuid.uuid1()): {
        "name": "CRW",
        "vip": True,
        "score": 0,
      },
      str(uuid.uuid1()): {
        "name": "DKG",
        "score": 0,
      },
      str(uuid.uuid1()): {
        "name": "JRG",
        "score": 0,
      },
      str(uuid.uuid1()): {
        "name": "DOG",
        "score": 0,
      },
    }

In [ ]:
response = stargate_client.put(players, DOC_ROOT_PATH + "/players")
pprint(response.json())

### Get the full game

In [ ]:
response = stargate_client.get({}, DOC_ROOT_PATH)
pprint(response.json())

#### Get just the players

In [ ]:
response = stargate_client.get({}, DOC_ROOT_PATH + "/players")
pprint(response.json())

### Change the game state to Tutorial

In [ ]:
response = stargate_client.put({"name": "TUTORIAL",
                                "roundId": ""}, 
                               DOC_ROOT_PATH + "/currentState")
pprint(response.json())

#### Get the current game state

In [ ]:
response = stargate_client.get({}, DOC_ROOT_PATH + "/currentState")
pprint(response.json())

### Move the game to round 1 input

In [ ]:
response = stargate_client.patch({"currentState": {"name": "ROUND_INPUT",
                                                    "roundId": 1}},
                                 DOC_ROOT_PATH)
pprint(response.json())

#### Get the current game state

In [ ]:
response = stargate_client.get({}, DOC_ROOT_PATH + "/currentState")
pprint(response.json())